# Moving car example by combining Speos files

This tutorial demonstrates how to run moving car workflow use case.
## Prerequisites

### Perform imports

In [1]:
import os
from pathlib import Path

from ansys.speos.core import Part, Speos, launcher
from ansys.speos.core.kernel.client import (
    default_docker_channel,
)
from ansys.speos.core.sensor import SensorCamera
from ansys.speos.core.simulation import SimulationInverse
from ansys.speos.core.source import SourceLuminaire
from ansys.speos.core.workflow.combine_speos import SpeosFileInstance, combine_speos


### Define constants
Constants help ensure consistency and avoid repetition throughout the example.

In [2]:
HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
CAR_NAMES = ["BlueCar", "RedCar"]
ENVIRONMENT_NAME = "Env_Simplified"
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.
USE_GPU = False

## Coordinate systems

Define the global coordinate systems for each of the assets.

In [3]:
GLOBAL_CS = [
    0,  # Origin x
    0,  # Origin y
    0,  # Origin z
    1,  # x-direction x
    0,  # x-direction y
    0,  # x-direction z
    0,  # y-direction x
    1,  # y-direction y
    0,  # y-direction z
    0,  # z-direction x
    0,  # z-direction y
    1,  # z-direction z
]
CAR_CS = {
    "red": [-4000, 0, 48000, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 0.0],
    "blue": [2000, 0, 35000, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
}

## Load assets
Assets used to run this example are available in the
[PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.

> **Note:** Make sure you
> have downloaded simulation assets and set ``assets_data_path``
> to point to the assets folder.

In [4]:
if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")

In [5]:
# ## Create connection with speos rpc server
if USE_DOCKER:
    speos = Speos(channel=default_docker_channel())
else:
    speos = launcher.launch_local_speos_rpc_server(port=GRPC_PORT)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/tools/common/cyberchannel.py:188: UserWarning: Starting gRPC client without TLS on localhost:50098. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


## Combine several speos files into one project

Here we are building a project with:
- An environment which is a road
- A blue car
- A red car

In [6]:
full_env_path = assets_data_path / f"{ENVIRONMENT_NAME}.speos" / f"{ENVIRONMENT_NAME}.speos"
car_paths = [assets_data_path / f"{car}.speos" / f"{car}.speos" for car in CAR_NAMES]
assets = [
    SpeosFileInstance(
        speos_file=str(full_env_path),
        axis_system=GLOBAL_CS,
    ),
    SpeosFileInstance(
        speos_file=str(car_paths[0]),
        axis_system=CAR_CS["red"],
    ),
    SpeosFileInstance(
        speos_file=str(car_paths[1]),
        axis_system=CAR_CS["blue"],
    ),
]
p = combine_speos(
    speos=speos,
    speos_to_combine=assets,
)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:760: UserWarning: The pySpeos feature : FaceStub.read_batch needs a Speos Version of 2025 R2 SP0 or higher.
  f_data_list = face_db.read_batch(refs=f_links)


In [7]:
print(p)

{
    "part_guid": "d12c6b9f-f06b-4eb3-ac8d-85b8a1b1dcc8",
    "materials": [
        {
            "name": "Env_Simplified.Material.1",
            "metadata": {
                "UniqueId": "c0011849-60dd-4eed-82fa-7c2b684eadd2"
            },
            "vop_guid": "1276c4f1-8d08-495b-a11a-8fb2dd525035",
            "geometries": {
                "geo_paths": [
                    "Env_Simplified/Solid:1195288205"
                ]
            },
            "sop_guid": "1667b293-c86e-4b1b-affd-6097cc162817",
            "description": "",
            "sop_guids": [],
            "vop": {
                "opaque": {},
                "name": "",
                "description": "",
                "metadata": {}
            },
            "sop": {
                "library": {
                    "sop_file_uri": "/app/assets/Env_Simplified.speos/RL_Road_107_6c27-753a-e5eb-38d0..anisotropicbsdf"
                },
                "name": "",
                "description": "",
         

## Preview the project

User can review the created/loaded project using preview method.

In [8]:
p.preview()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html lang=&quot;en-US&quot; dir=&quot;ltr&quot;>\n  …

## Complete the project with sensor/source/simulation

We are adding a camera sensor to have output results, a luminaire to have a light source.

And, we gather the source and the sensor into a simulation (we will compute it just after).

### Create a sensor

In [9]:
ssr = p.create_sensor(name="Camera.1", feature_type=SensorCamera)
ssr.set_distortion_file_uri(
    uri=str(assets_data_path / "CameraInputFiles" / "CameraDistortion_190deg.OPTDistortion")
).set_mode_photometric().set_transmittance_file_uri(
    uri=str(assets_data_path / "CameraInputFiles" / "CameraTransmittance.spectrum")
).set_mode_color().set_red_spectrum_file_uri(
    uri=str(assets_data_path / "CameraInputFiles" / "CameraSensitivityRed.spectrum")
).set_blue_spectrum_file_uri(
    uri=str(assets_data_path / "CameraInputFiles" / "CameraSensitivityBlue.spectrum")
).set_green_spectrum_file_uri(
    uri=str(assets_data_path / "CameraInputFiles" / "CameraSensitivityGreen.spectrum")
)
ssr.set_axis_system([-2000, 1500, 11000, -1, 0, 0, 0, 1, 0, 0, 0, -1])
ssr.commit()

### Create a source

In this example, a luminaire source is created with an IES file.

More details on creating/editing source examples can be found in core examples.

In [10]:
src = p.create_source(name="Luminaire.1", feature_type=SourceLuminaire)
src.set_intensity_file_uri(
    uri=str(assets_data_path / "IES_C_DETECTOR.ies")
).set_spectrum().set_daylightfluorescent()
src.set_axis_system([0, 10000, 50000, 1, 0, 0, 0, 1, 0, 0, 0, 1])

src.commit()

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:224: UserWarning: The pySpeos feature : SourceLuminaire needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SourceLuminaire(


### Create a simulation

More details on creating/editing simulation examples can be found in core examples.

In [11]:
sim = p.create_simulation(name="Inverse.1", feature_type=SimulationInverse)
sim.set_sensor_paths(["Camera.1"]).set_source_paths(["Luminaire.1"])
sim.commit()

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:309: UserWarning: The pySpeos feature : SimulationInverse needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SimulationInverse(


## Run the simulation

Simulation can be run using CPU via compute_CPU method or using GPU via compute_GPU method.

In [12]:
run_sim = sim.compute_GPU if USE_GPU else sim.compute_CPU
run_sim()  # Run the simulation

[upload_response {
  info {
    uri: "4d76561b-014e-4ef0-a7fe-b5a0e0bd2c93"
    file_name: "Camera.1.Irradiance.xmp"
    file_size: 10638901
  }
  upload_duration {
    nanos: 9718974
  }
}
, upload_response {
  info {
    uri: "fe15dde5-2279-41c5-a009-226533c7435c"
    file_name: "Camera.1.hdr"
    file_size: 518993
  }
  upload_duration {
    nanos: 756465
  }
}
, upload_response {
  info {
    uri: "16cb7512-bf0b-4a82-b4ba-e0209ff4478f"
    file_name: "Camera.1.xmp"
    file_size: 1805085
  }
  upload_duration {
    nanos: 2094709
  }
}
, upload_response {
  info {
    uri: "cd84e4d9-4316-4c35-a1ad-4734c3ec7023"
    file_name: "Camera.1.png"
    file_size: 741460
  }
  upload_duration {
    nanos: 969092
  }
}
, upload_response {
  info {
    uri: "9a90be04-b55a-4e2b-a6f4-81fccb249ef5"
    file_name: "Inverse.1.html"
    file_size: 997663
  }
  upload_duration {
    nanos: 1266876
  }
}
]

## Check and review result

Open result (only windows)

In [13]:
if os.name == "nt":
    from ansys.speos.core.workflow.open_result import open_result_image

    open_result_image(simulation_feature=sim, result_name="Camera.1.png")

## Modify part

Move the part via changing the axis_system of a part.

axis_system is a list of 12 float values:
x, y, z,
x_vect_x, x_vect_y, x_vect_z,
y_vect_x, y_vect_y, y_vect_z,
z_vect_x, z_vect_y, z_vect_z.

In [14]:
blue_car_sub_part = p.find(name="BlueCar", feature_type=Part.SubPart)[0]
blue_car_sub_part.set_axis_system([2000, 0.0, 20000, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 1.0, 0.0])
blue_car_sub_part.commit()

## Re-run simulation with the modified part position

In [15]:
run_sim()

[upload_response {
  info {
    uri: "93060d2e-21eb-4dd0-b41d-5f24b481e616"
    file_name: "Camera.1.Irradiance.xmp"
    file_size: 10378477
  }
  upload_duration {
    nanos: 10375625
  }
}
, upload_response {
  info {
    uri: "128cffe4-2aa0-445c-963b-0bc82ab6a849"
    file_name: "Camera.1.hdr"
    file_size: 519948
  }
  upload_duration {
    nanos: 833715
  }
}
, upload_response {
  info {
    uri: "4aaf1825-4b6e-493c-996d-1cfc25af4c33"
    file_name: "Camera.1.xmp"
    file_size: 1762835
  }
  upload_duration {
    nanos: 2083451
  }
}
, upload_response {
  info {
    uri: "6e161104-2d71-4a37-928f-524a9fad3c00"
    file_name: "Camera.1.png"
    file_size: 728543
  }
  upload_duration {
    nanos: 874149
  }
}
, upload_response {
  info {
    uri: "f902ce27-e691-4e34-87e2-c85276c4da15"
    file_name: "Inverse.1.html"
    file_size: 981592
  }
  upload_duration {
    nanos: 1133928
  }
}
]

Review result:

In [16]:
if os.name == "nt":
    open_result_image(simulation_feature=sim, result_name="Camera.1.png")

## Modify camera property

Modify the camera, e.g. focal length to 10

In [17]:
cam1 = p.find(name="Camera.1", feature_type=SensorCamera)[0]
cam1.set_focal_length(value=10)
cam1.commit()

Re-run the simulation and review result

In [18]:
run_sim()
if os.name == "nt":
    open_result_image(simulation_feature=sim, result_name="Camera.1.png")

In [19]:
speos.close()

True